# KAN med service access data
Idee hentet fra https://github.com/team-daniel/KAN/blob/master/KAN_classification.ipynb


In [17]:
# Refresh dependencies
#!pip install -r ../requirements.txt

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

df = pd.read_csv('service_access_data/Generated_Access_Log_Dataset.csv')
df.head()

,username,geography,weekday,servicename,classification
0,Nils,Hvaler,Saturday,userprofile,ok
1,Ola,Oslo,Saturday,login,ok
2,Nils,Hvaler,Monday,login,ok
3,Nina,Bergen,Friday,userprofile,ok
4,Nina,Trondhjem,Tuesday,order,ok


# Beskrivelse av datasettet
Dette er en tilgangslogg til tjenester i en butikk.
Dette skal klassifisere forsøk på tjenestetilgang i tre klasser:
 * ok
 * error
 * suspicious

Innput er 4 parametre:
 * username
 * geography
 * weekday
 * servicename

Servicenames:
 * login
 * userprofile
 * userreview
 * order

Geography er en forenkling av lokasjon til IP-adresse:
 * Hvaler
 * Bergen
 * Trondhjem
 * Oslo
 * Fredrikstad

Usernames:
 * Nils, reiser mye, og kjøper mye. Legger aldri inn userreview
 * Ola, kjøper litt. Bruker butikken kun fra Oslo og Hvaler. Legger kun inn userreview på søndager, fra Oslo
 * Per, kan finne på å missbruke Nils og Ola sine logins. Bruker butikken kun fra Bergen
 * Hilde, bruker butikken fra Fredrikstad, Oslo og Hvaler. Random bruk av tjenester og dager.
 * Nina, bruker butikken fra Bergen og Trondhjem. Random bruk av tjenester og dager.

## login.target_names
Oversikt over det tre blomstertypene i datasettet. Kalles klasser, og oversettes til 0, 1 eller 2 i iris.target
np.ndarray(['ok', 'failed', 'suspicious'], dtype='<U10')

## login.target
Hver rad er en , og kolonnen er hvilken klasse login tilhører. 0, 1 eller 2

## iris.feature_names
Beskriver hva de fire kolonnene i iris.data er. ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

## iris.data
Hver rad er en blomst, og kolonnene er sepal length, sepal width, petal length og petal width.
150x4 numpy.ndarray.

In [19]:
feature_names = ['username', 'geography', 'weekday', 'servicename']
# Konvertere kategoriske variabler til numeriske
label_encoders = {}
for column in feature_names:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Definere input og output
X = df[feature_names]
y = df['classification']

# Splitte data i trenings- og testsett
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# login.target_names, Tjenestetilgangsklasser
unique_values = np.unique(y)
print("iris.target_names")
print("Tjenestetilgangsklasser:", unique_values)

iris.target_names
Tjenestetilgangsklasser: ['error' 'ok' 'suspicious']


In [21]:
print("iris.target")
y

iris.target


0         ok
1         ok
2         ok
3         ok
4         ok
       ...  
995       ok
996       ok
997       ok
998       ok
999    error
Name: classification, Length: 1000, dtype: object

In [22]:
# login.feature_names, Input parametre
print("iris.feature_names")
feature_names

iris.feature_names


['username', 'geography', 'weekday', 'servicename']

In [23]:
print("iris.data")
X

iris.data


,username,geography,weekday,servicename
0,1,2,2,2
1,3,3,2,0
2,1,2,1,0
3,2,0,0,2
4,2,4,5,1
...,...,...,...,...
995,4,0,4,1
996,3,3,0,1
997,2,4,0,3
998,0,2,5,0


***
# RandomForestClassifier

In [24]:
# Velge og trene modellen
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predikere og evaluere modellen
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


              precision    recall  f1-score   support

       error       0.08      0.04      0.06        23
          ok       0.81      0.92      0.86       154
  suspicious       0.75      0.39      0.51        23

    accuracy                           0.76       200
   macro avg       0.54      0.45      0.48       200
weighted avg       0.72      0.76      0.73       200

Accuracy: 0.755


***
# Prediksjon

In [26]:
# Ny serviceforespørsel
new_request = {
    'username': 'Ola',
    'geography': 'Oslo',
    'weekday': 'Sunday',
    'servicename': 'userreview'
}

# Konvertere den nye forespørselen til numeriske verdier ved å bruke de opprinnelige label encoders
new_request_encoded = {}
for column in feature_names:
    encoded_value = label_encoders[column].transform([new_request[column]])[0]
    new_request_encoded[column] = [encoded_value]

# Gjøre prediksjon
new_request_df = pd.DataFrame(new_request_encoded)  # Bruke en DataFrame for prediksjon
prediction = model.predict(new_request_df)

# Skriv ut prediksjonen
print(f'The classification for the new request is: {prediction[0]}')


The classification for the new request is: ok


## Suspect


In [27]:
new_request = {
    'username': 'Ola',
    'geography': 'Bergen',
    'weekday': 'Wednesday',
    'servicename': 'userreview'
}

# Konvertere den nye forespørselen til numeriske verdier ved å bruke de opprinnelige label encoders
new_request_encoded = {}
for column in feature_names:
    encoded_value = label_encoders[column].transform([new_request[column]])[0]
    new_request_encoded[column] = [encoded_value]

# Gjøre prediksjon
new_request_df = pd.DataFrame(new_request_encoded)  # Bruke en DataFrame for prediksjon
prediction = model.predict(new_request_df)

# Skriv ut prediksjonen
print(f'The classification for the new request is: {prediction[0]}')

The classification for the new request is: suspicious
